+ Dataset : http://gigadb.org/dataset/100542
+ ログ込みは : https://gist.github.com/s-n-1-0/18e402a7fefe69be739f0a7e8fdfa0d8

## 設定

In [ ]:
import scipy.io
import numpy as np
dataset_size = (54,54)
root_path = "//172.16.88.200/private/2221012/MIOnly_FTP_EEG Dataset and OpenBMI Toolbox for Three BCI Paradigms"
train_key = "EEG_MI_train"
test_key = "EEG_MI_test"

## まずはファイル単体で処理(preview)

In [ ]:
data_mat = scipy.io.loadmat(root_path + "/session1/sess01_subj01_EEG_MI.mat")
data_mat.keys()

In [ ]:
def parse_mat(parent_key,mat_data):
    data = mat_data[parent_key][0][0]
    #形式整理
    for i in range(len(data)):
        sq_data = data[i].squeeze()
        if len(sq_data.shape) == 1:
            data[i] =  sq_data
        if len(sq_data.shape) == 0:
            data[i] = sq_data
    return data

train = parse_mat(train_key,data_mat)
test = parse_mat(test_key,data_mat)
[ print(t.shape) for t in train];
print("※()は1x1の値")

各要素  

0. smt: 試行ごとに分割された信号
1. x : 分割されていない信号
2. t : 分割のインデックス
3. fs : サンプリングレート
4. y_dec : 試行のラベル(1 or 2)
5. y_logic : y_decのワンホットラベル版
6. y_class : y_decの"right" : "left"版
7. class : 値とラベルの対応表
8. chan : 使用しているチャンネルのリスト
9. ival : インターバルの連番インデックス
10. time_interval : 1試行の間隔タプル(0,4000)
11. EMG : EMG信号
12. EMG_index : EMGの対応表
13. pre_rest : 1分間の休憩(測定前)信号
14. post_rest : 1分間の休憩(測定後)信号
15. rest_interval : 休憩の間隔タプル(0,60000)

In [ ]:
print([t[0] for t in train[8]])

In [ ]:
# 使うのはsmtとy_decのみ
def extract_data(parsed_data):
     return parsed_data[0],parsed_data[4]
print(f"X :{np.concatenate([extract_data(d)[0] for d in [train,test]],axis=1).shape}")
print(f"Y :{np.concatenate([extract_data(d)[1] for d in [train,test]]).shape}")

## まとめて処理

In [ ]:
def save_npy(name,extracted_data):
    np.save(f"{root_path}/{name}_x",extracted_data[0])
    np.save(f"{root_path}/{name}_y",extracted_data[1])
for i, size in enumerate(dataset_size):
    for j in range(size):
        session = scipy.io.loadmat(f"{root_path}/session{i+1}/sess{(i+1):02}_subj{(j+1):02}_EEG_MI.mat")
        train = extract_data(parse_mat(train_key,session))
        test = extract_data(parse_mat(test_key,session))
        save_npy(f"prev_dataset/s{(i+1)}_subj{j+1}_train",train)
        save_npy(f"prev_dataset/s{(i+1)}_subj{j+1}_test",test)